In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from datetime import datetime, timedelta
import time
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError as dke

In [ ]:
driver = webdriver.Chrome(executable_path='chromedriver.exe')
driver.get('https://mail.ru/')

#Ожидание от сервера
driver.implicitly_wait(10)

# Ввод логина
elem = driver.find_element(By.CLASS_NAME, 'email-input')
elem.send_keys('study.ai_172')

elem.send_keys(Keys.ENTER)
# Ввод пароля
elem = driver.find_element(By.CLASS_NAME, 'password-input')
elem.send_keys('NextPassword172#') 

elem.send_keys(Keys.ENTER)

messages = []
actions = ActionChains(driver)

i = 0
last_item = ''
while True:
    
    items = driver.find_elements(By.XPATH, "//a[contains(@class,'js-letter-list-item')]")
    
    if items[-1] == last_item: # проверяю совпадает ли последний эл-нт, с предедущим циклом
        i += 1
        if i == 4: # Для перестраховки, если 4 раза совпала, выходим из цикла
            break
    else: # если вдруг не проскролилось, а страница не закончилась, обнуляю
        i = 0
    
    for item in items:
        message = {}
        _id = item.get_attribute('data-uidl-id')
        link = item.get_attribute('href')
        date_of_letter = item.find_element(By.CLASS_NAME, 'llc__item_date').get_attribute('title')
        letter_subject = item.find_element(By.CLASS_NAME, 'll-sj__normal').text

        message['_id'] = _id
        message['link'] = link
        message['date_of_letter'] = date_of_letter
        message['letter_subject'] = letter_subject 

        messages.append(message)
        
    actions.move_to_element(items[-1])
    actions.perform()
    
    last_item = items[-1] # сохраняю последний элемент
    
    time.sleep(5)

<ipython-input-2-68f1a8dbde2d>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='chromedriver.exe')


In [ ]:
# Собираю текст сообщения
for message in messages:
    driver.get(message['link'])
    message['letter_text'] = driver.find_element(By.CLASS_NAME, 'letter-body').text

In [137]:
# Добавление в базу данных MongoDB
client = MongoClient('127.0.0.1', 27017)
db = client['mail']
letters = db.letters

In [218]:
months = ["Unknown", "Января", "Февраля", "Марта", "Апреля", "Мая",
          "Июня", "Июля", "Августа", "Сентября", "Октября",
          "Ноября", "Декабря"]
yesterday = datetime.now() - timedelta(1)
month = (months[datetime.now().month])
datetime.strftime(yesterday, f'%d {month}')
for letter in messages:
    if letter['date_of_letter'].split(' ')[0] == 'Сегодня,':
        letter['date_of_letter'] = ''.join([letter['date_of_letter'].split(' ')[0]
                 .replace('Сегодня,', datetime.strftime(datetime.now(), f'%d {month}, ')), 
                 letter['date_of_letter'].split(' ')[1]])
    elif letter['date_of_letter'].split(',')[0] == 'Вчера':
        letter['date_of_letter'] = ''.join([letter['date_of_letter'].split(',')[0]
                 .replace('Вчера', datetime.strftime(yesterday, f'%d {month}, ')),
                 letter['date_of_letter'].split(',')[1]])
    try:
        letters.insert_one(letter)
    except dke:
        print(f"Вакансия с id {letter['_id']} уже добавлена")

Вакансия с id 16365722171360693032 уже добавлена
Вакансия с id 16365670231244638115 уже добавлена
Вакансия с id 16365657361767513173 уже добавлена
Вакансия с id 16365634201505627959 уже добавлена
Вакансия с id 16365619191136570781 уже добавлена
Вакансия с id 16365611791385982306 уже добавлена
Вакансия с id 16365599790348551025 уже добавлена
Вакансия с id 16365575351889347922 уже добавлена
Вакансия с id 16365425070646016931 уже добавлена
Вакансия с id 16365356762139391785 уже добавлена
Вакансия с id 16365302000449317994 уже добавлена
Вакансия с id 16364922500173317391 уже добавлена
Вакансия с id 16364893961504034563 уже добавлена
Вакансия с id 16365722171360693032 уже добавлена
Вакансия с id 16365670231244638115 уже добавлена
Вакансия с id 16365657361767513173 уже добавлена
Вакансия с id 16365634201505627959 уже добавлена
Вакансия с id 16365619191136570781 уже добавлена
Вакансия с id 16365611791385982306 уже добавлена
Вакансия с id 16365599790348551025 уже добавлена
Вакансия с id 163655